In [ ]:
hill1=cv2.cvtColor(cv2.imread("/content/hill/1.JPG"),cv2.COLOR_BGR2RGB)
hill1=cv2.imread("/content/hill/1.JPG")
hill2=cv2.cvtColor(cv2.imread("/content/hill/2.JPG"),cv2.COLOR_BGR2RGB)
hill3=cv2.cvtColor(cv2.imread("/content/hill/3.JPG"),cv2.COLOR_BGR2RGB)

TV1=cv2.cvtColor(cv2.imread("/content/tv/1.jpg"),cv2.COLOR_BGR2RGB)
TV2=cv2.cvtColor(cv2.imread("/content/tv/2.jpg"),cv2.COLOR_BGR2RGB)
TV3=cv2.cvtColor(cv2.imread("/content/tv/3.jpg"),cv2.COLOR_BGR2RGB)
import cv2
import copy

def drawMatches(image1, kp1, image2, kp2, idx_pairs):
    keypt1 = [cv2.KeyPoint(coord[1], coord[0], 40) for coord in kp1.tolist()]
    keypt2 = [cv2.KeyPoint(coord[1], coord[0], 40) for coord in kp2.tolist()]
    matches = [cv2.DMatch(pair[0], pair[1], 0) for pair in idx_pairs.tolist()]
    return cv2.drawMatches(image1, keypt1, image2, keypt2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

def plot_matches(images, feature_coord, matches, img_idx):
    matched_img = drawMatches(images[img_idx], feature_coord[img_idx], images[img_idx-1], 
                              feature_coord[img_idx-1], matches[img_idx-1])
    cv2.imshow('Matches Found', matched_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def est_homography(src, dest):
    N = src.shape[0]
    if N != dest.shape[0]:
        raise ValueError("src and diff should have the same dimension")
    src_h = np.hstack((src, np.ones((N, 1))))
    A = np.array([np.block([[src_h[n], np.zeros(3), -dest[n, 0] * src_h[n]],
                            [np.zeros(3), src_h[n], -dest[n, 1] * src_h[n]]])
                  for n in range(N)]).reshape(2 * N, 9)
    [_, _, V] = np.linalg.svd(A)
    return V.T[:, 8].reshape(3, 3)

def apply_homography(H, src):
    src_h = np.hstack((src, np.ones((src.shape[0], 1))))
    dest =  src_h @ H.T
    return (dest / dest[:,[2]])[:,0:2]
def normalize(x, y):
  n=x.shape[0]
  x_mean=np.mean(x)
  y_mean= np.mean(y)
  x_std= np.std(x)
  y_std=np.std(y)
  T=np.array([[1/x_std,0,0],[0,1/y_std,0],[0,0,1]])*np.array([[1,0,-x_mean],[0,1,-y_mean],[0,0,1]])
  print(np.shape(T))
  Normal=[]
  for i in range(n):
    X=np.matmul(T,np.array([x[i],y[i],1],dtype=object))
    Normal.append([X[0][0],X[1][0],X[2][0]])
  Normal=np.array(Normal)
  x1_n=Normal[:,0]
  y1_n=Normal[:,1]

  return x1_n,y1_n,T
def ransac(image1,image2):
  M1,M2= Keypoint_Matching(image1,image2)
  maximum_numberof_inliers=0
  for i in range(8000):
    random_index= np.random.choice(M1.shape[0],4)
    # random.shuffle(random_index)
    H=est_homography(M1[random_index],M2[random_index])
    M2_prime=apply_homography(H,M1)
    D=np.zeros((len(M1)))
    D=np.sqrt((M2_prime-M2)**2).sum(axis=1)
    numberof_inliers=np.count_nonzero(D[D<3])
    if numberof_inliers>maximum_numberof_inliers:
      maximum_numberof_inliers=numberof_inliers
      H_final=copy.deepcopy(H)
  return H_final 

def getKeyPoints(image):
  Gray_image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  dst = cv2.cornerHarris(Gray_image,2,3,0.04)
  dst = cv2.dilate(dst,None)
  ret, dst = cv2.threshold(dst,0.01*dst.max(),255,0)
  dst = np.uint8(dst)
  # find centroids
  ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)
  # define the criteria to stop and refine the corners
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
  corners = cv2.cornerSubPix(Gray_image,np.float32(centroids),(5,5),(-1,-1),criteria)
  res = np.hstack((centroids,corners))
  res = np.int0(res)
  return res

def Sift_keypts(image):
  Gray_image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  sift=cv2.SIFT_create()
  point, descriptor = sift.detectAndCompute(Gray_image,None)
  pts=np.int0(cv2.KeyPoint_convert(point))
  return pts, descriptor

def Keypoint_Matching(image1,image2):

  #Matching keypoints using SIFT

  keypoint1,Descriptor1 =Sift_keypts(image1)
  keypoint2, Descriptor2= Sift_keypts(image2)
  Distance= distance_matrix(Descriptor1,Descriptor2)
  ith_neighbors=np.argsort(Distance,axis=1)[:,:2]
  Neighbors=np.take_along_axis(Distance,ith_neighbors,axis=1)
  Z= Neighbors[:,0]/Neighbors[:,1]
  thresh=0.7
  matches=np.column_stack((np.arange(Descriptor1.shape[0]), ith_neighbors[:,0]))
  ith_match=matches[Z<thresh,:]
  matched_keypoint1=keypoint1[ith_match[:,0]]
  matched_keypoint2=keypoint2[ith_match[:,1]]
  M1=matched_keypoint1
  M2=matched_keypoint2

  return M1,M2

def Warping(x_min,y_min,image,M):
  H=np.int0(np.round(np.abs(y_min)+image.shape[0]))
  W=np.int0(np.round(np.abs(x_min)+image.shape[1]))
  size=(W,H)
  return cv2.warpPerspective(src=image,M=M,dsize=size)

def image_stitch(image1,image2,H):
  image1=cv2.normalize(image1.astype('float'),None,0.0,1.0,cv2.NORM_MINMAX)
  image2=cv2.normalize(image2.astype('float'),None,0.0,1.0,cv2.NORM_MINMAX)
  h,w,colors=image1.shape
  Y=np.array([[0,0,1],[w,0,1],[0,w,1],[w,h,1]])
  P=np.dot(H,Y.T)
  y1=min(P[1]/P[2])
  x1=min(P[0]/P[2])
  Trans=np.array([[1,0,-x1],[0,1,-y1],[0,0,1]])
  Warp2=Warping(x1,y1,image2,Trans)
  H=np.dot(Trans,H)
  Warp1=Warping(x1,y1,image1,H)
  for idx in range(Warp2.shape[0]):
    for jdx in range(Warp2.shape[1]):
      if (Warp1[idx][jdx]==[0,0,0]).all() and (Warp2[idx][jdx]!=[0,0,0]).all():
        Warp1[idx,jdx,:]=Warp2[idx,jdx,:]
  Finally=Warp1[:Warp2.shape[0],:Warp2.shape[1],:]
  return Finally
#Hill Stitching

M1,M2=Keypoint_Matching(hill1,hill2)
H_1=ransac(hill1,hill2)
S1=image_stitch(hill1,hill2,H_1) 
# plt.imshow(S)
S1=cv2.normalize(S1,None,0,255,cv2.NORM_MINMAX)
cv2.imwrite('S1.jpg',S1)
S12=cv2.cvtColor(cv2.imread("/content/S1.jpg"),cv2.COLOR_BGR2RGB)
H_2= ransac(S12,hill3) 
S2=image_stitch(S1,hill3,H_2)
print("H matrix for hill",H_2) 
plt.imshow(S2)


#TV stitching
M1_TV,M2_TV=Keypoint_Matching(TV1,TV2)
H_TV=ransac(TV1,TV2)
S1_TV=image_stitch(TV1,TV2,H_TV) 
# plt.imshow(S)
S1_TV=cv2.normalize(S1_TV,None,0,255,cv2.NORM_MINMAX)
cv2.imwrite('S1_TV.jpg',S1_TV)
S12_TV=cv2.cvtColor(cv2.imread("/content/S1_TV.jpg"),cv2.COLOR_BGR2RGB)
H_2_TV= ransac(S12_TV,TV3) 
S2_TV=image_stitch(S12_TV,TV3,H_2_TV)
print("H matrix for TV",H_2_TV) 
plt.imshow(S2_TV)
